In [ ]:
# !pip install transformers[sentencepiece]

## BART zero-shot Text Classification

In [1]:
from transformers import pipeline 
import pandas as pd 
classifier = pipeline("zero-shot-classification", 
                      model="facebook/bart-large-mnli") 
sequence_to_classify = "one day I will see the world" 
candidate_labels = ['travel', 
                    'cooking', 
                    'dancing', 
                    'exploration'] 
result = classifier(sequence_to_classify, candidate_labels) 
pd.DataFrame(result)

,sequence,labels,scores
0,one day I will see the world,travel,0.795757
1,one day I will see the world,exploration,0.199331
2,one day I will see the world,dancing,0.002621
3,one day I will see the world,cooking,0.002291


In [2]:
result = classifier(sequence_to_classify,  
                      candidate_labels,  
                      multi_label=True) 
pd.DataFrame(result) 

,sequence,labels,scores
0,one day I will see the world,travel,0.994511
1,one day I will see the world,exploration,0.938388
2,one day I will see the world,dancing,0.005706
3,one day I will see the world,cooking,0.001819


## BART no pipeline
BART is fine-tuned on Natural Language Inference (NLI) datasets such as MNLI. These datasets contain sentence pairs and three classes for each pair; that is, *Neutral*, *Entailment*, and *Contradiction*. Models that have been trained on these datasets can capture the semantics of two sentences and classify them by assigning a label in one-hot format.   
If you take out Neutral labels and only use *Entailment* and *Contradiction* as your output labels, if two sentences can come after each other, then it means these two are closely related to each other. In other words, you can change the first sentence to the label (travel, for example) and the second sentence to the content (one day I will see the world, for example).  
According to this, if these two can come after each other, this means that the label and the content are semantically related. The following code example shows how to directly use the BART model without the zero-shot classification pipeline according to the preceding descriptions:

In [3]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer 

nli_model = AutoModelForSequenceClassification.from_pretrained("facebook/bart-large-mnli") 
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-mnli") 

In [9]:
premise = "one day I will see the world" 
# label = "travel" 
label = "retirement"
hypothesis = f'This example is {label}.' 

In [10]:
x = tokenizer.encode( 
    premise, 
    hypothesis, 
    return_tensors='pt', 
    truncation_strategy='only_first') 

/home/guy/anaconda3/envs/mastrans/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:2356: FutureWarning: The `truncation_strategy` argument is deprecated and will be removed in a future version, use `truncation=True` to truncate examples to a max length. You can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to truncate to the maximal input size of the model (e.g. 512 for Bert).  If you have pairs of inputs, you can give a specific truncation strategy selected among `truncation='only_first'` (will only truncate the first sentence in the pairs) `truncation='only_second'` (will only truncate the second sentence in the pairs) or `truncation='longest_first'` (will iteratively remove tokens from the longest sentence in the pairs).
  warnings.warn(


In [11]:
x

tensor([[   0, 1264,  183,   38,   40,  192,    5,  232,    2,    2,  713, 1246,
           16, 3832,    4,    2]])

In [12]:
logits = nli_model(x)[0] 
logits

tensor([[ 1.1138,  3.1326, -4.1702]], grad_fn=<AddmmBackward0>)

In [13]:
entail_contradiction_logits = logits[:,[0,2]]   # take the logits for entailment and contradiction
probs = entail_contradiction_logits.softmax(dim=1) 
prob_label_is_true = probs[:,1] 
print(prob_label_is_true) 

tensor([0.0050], grad_fn=<SelectBackward0>)
